Establish connection

In [1]:
%reload_ext google.cloud.bigquery
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="C:\\Users\\Cermati\\AppData\\Roaming\\gcloud\\application_default_credentials.json"
print(os.environ["GOOGLE_APPLICATION_CREDENTIALS"])
from google.cloud import bigquery

C:\Users\Cermati\AppData\Roaming\gcloud\application_default_credentials.json


Set Date Parameter & Version

In [2]:
%%bigquery param1 --project athena-179008
SELECT DATE_ADD(DATE_TRUNC(CURRENT_DATE,MONTH),INTERVAL -1 MONTH) as dt

In [3]:
param_date=str(param1['dt'].iloc[0])
version = '1'
print(param_date)
print(version)

2021-10-01
1


Get Data ContractData to Dataframe contractData

In [4]:
client = bigquery.Client(project="athena-179008")
sql = """
    /* Define Data mart Period.*/
with report_period as (
    select 
        date_trunc(date('{Period}'),month) as period
        , case 
            when date('{Period}') < '2019-08-01' then date_add(date_trunc(date('{Period}'),month),interval 1 month)
            else date_add(date_add(date_trunc(date('{Period}'),month),interval 1 month),interval -1 day) 
        end as dm_period
        , date_add(date_trunc(date('{Period}'),month),interval 1 month) as snapshot_date
)
/*CONTRACT ATH*/
, contracts_ath as (
    select 
        lc.dataDate
        , coalesce(c.applicationOrderId,a.orderId) as applicationOrderId
        , coalesce(c.id,lc.id) as id
        , lc.id as loan_contract_id
        , lc.status
        , lc.approved_date as approvedDate
        , lc.paid_off_date as paidOffDate
        , cast(lc.loan_amount as int64) as approvedLoanAmount
        , cast(lc.principal as int64) as principal
        , lc.tenure as approvedLoanTenureInMonth
        , coalesce(lcl.dpd,0) as dayPastDue
        , case when lcl.max_dpd_paid is null then lcl.dpd when lcl.dpd is null then lcl.max_dpd_paid else greatest(lcl.dpd,lcl.max_dpd_paid) end as maxDayPastDue
        , lcl.nextduedate
        , cast(lcl.balance as int64) as balance
        , cast(lcl.balanceWithInterest as int64) as balanceWithInterest
        , cast(lcl.latefee as int64) as latefee
        , lcl.last_due_date
        , lc.type
        , r.interest_rate
    from athena-179008.vayu_data_mart.indodana_credit_loan_contracts as lc
    inner join data-platform-indodana.vayu.indodana_credit_installment_requests as ir on lc.installment_request_id = ir.id
    inner join data-platform-indodana.vayu.indodana_athena_applications a on a.orderId=ir.entity_id
    LEFT join (
        select 
            dataDate
            , loan_contract_id
            , min(case when status in ('UNPAID','PARTIALLY_PAID') then due_date end) as nextduedate
            , date_diff(dataDate,date(min(case when status in ('UNPAID','PARTIALLY_PAID') then due_date end),"Asia/Jakarta"),day) as dpd
            , max(case when status = 'PAID' then date_diff(date(paid_off_date,"Asia/Jakarta"),date(due_date,"Asia/Jakarta"),day) end) max_dpd_paid
            , sum(case when type='PRINCIPAL' then balance else 0 end) as balance
            , sum(case when type IN ('INTEREST','PRINCIPAL') then balance else 0 end) as balanceWithInterest
            , sum(case when type='LATE_FEE' then balance else 0 end) as latefee
            , max(case when type='PRINCIPAL' then due_date end) as last_due_date
        from athena-179008.vayu_data_mart.indodana_credit_loan_contract_ledgers 
        where status not in ('CANCELLED')
        group by 1,2
    ) lcl on lcl.loan_contract_id=lc.id AND lc.dataDate=lcl.dataDate
    LEFT JOIN data-platform-indodana.vayu.indodana_credit_credit_rates r ON r.id=lc.credit_rate_id
    LEFT JOIN data-platform-indodana.vayu.indodana_athena_contracts c ON c.creditLoanContractId=lc.id
    WHERE ir.type IN ('CASH_LOAN')
)
/* Get All contractId that has payments during reporting period. */
, payment_current_period as (
    select p.loan_contract_id
        , max(Provider_Received_Date) as max_payment_date
    from athena-179008.vayu_data_mart.view_ath_user_payments p
    where Provider_Received_Date < (select max(approvedDate) from contracts_ath where dataDate=(select dm_period from report_period))
    and date(Provider_Received_Date,"Asia/Jakarta") >= (select period from report_period)
    group by 1
    
)
/*Data CLI*/
, ledgers as (
    select 
        dataDate
        , loan_contract_id
        , min(case when status in ('UNPAID','PARTIALLY_PAID') then due_date end) as next_due_date
        , date_diff(dataDate,date(min(case when status in ('UNPAID','PARTIALLY_PAID') then due_date end),"Asia/Jakarta"),day) as dpd
        , max(case when status = 'PAID' then date_diff(date(paid_off_date,"Asia/Jakarta"),date(due_date,"Asia/Jakarta"),day) end) max_dpd_paid
        , sum(balance) as outstanding_balance
        , sum(case when type='PRINCIPAL' then balance else 0 end) as balance_principal
        , sum(case when type='INTEREST' then balance else 0 end) as balance_interest
        , sum(case when type='LATE_FEE' then balance else 0 end) as balance_latefee
    from athena-179008.vayu_data_mart.indodana_credit_loan_contract_ledgers 
    , report_period rp
    where status not in ('CANCELLED')
    and dataDate=rp.dm_period
    group by 1,2
)
, trx as (
    select 
        ir.borrower_id
        , a.OrderId
        , a.partner
        , ir.credit_limit_account_id
        , ir.entity_id
        , lc.id as loan_contract_id
        , CASE WHEN f.id IS NOT NULL then 'CANCELLED' else lc.status END as status
        , lc.tenure
        , lc.approved_date
        , lc.paid_off_date
        , cast(lc.loan_amount as int64) as loan_amount
        , cast(lc.principal as int64) as principal
        , cast(lc.interest as int64) as interest
        , cast((lc.principal+lc.interest)/lc.tenure as int64) as installment
        , lcl.next_due_date
        , coalesce(lcl.dpd,0) as dpd
        , case when lcl.max_dpd_paid is null then lcl.dpd when lcl.dpd is null then lcl.max_dpd_paid else greatest(lcl.dpd,lcl.max_dpd_paid) end as max_dpd
        , cast(lcl.outstanding_balance as int64) as outstanding_balance
        , cast(lcl.balance_principal as int64) as balance_principal
        , cast(lcl.balance_interest as int64) as balance_interest
        , cast(lcl.balance_latefee as int64) as balance_latefee
        , dm_period
        , snapshot_date
        , r.interest_rate
    from athena-179008.vayu_data_mart.indodana_credit_loan_contracts as lc
    inner join data-platform-indodana.vayu.indodana_credit_installment_requests as ir on lc.installment_request_id = ir.id
    inner join data-platform-indodana.vayu.indodana_credit_credit_limit_accounts cla on cla.id=ir.credit_limit_account_id
    LEFT join data-platform-indodana.vayu.indodana_athena_applications app
        ON coalesce(app.partner,'INDODANA')=cla.type
        AND app.userId=ir.borrower_id
        and substr(app.orderId,1,3)='CLI'
        and app.status='Approved'
    inner join data-platform-indodana.vayu.indodana_athena_applications a 
        on a.orderId=COALESCE(SPLIT(cla.idempotency_key,'_application.')[SAFE_OFFSET(1)],app.OrderId)
    inner join ledgers lcl on lcl.loan_contract_id=lc.id
    LEFT JOIN data-platform-indodana.vayu.indodana_credit_credit_rates r ON r.id=lc.credit_rate_id
    /* Check Fraud Victim Data */
    LEFT JOIN (
        SELECT
            lc.id, cla.updated_At
        FROM athena-179008.vayu_data_mart.indodanamarketing_cli_application cli
        INNER JOIN data-platform-indodana.vayu.indodana_credit_credit_limit_accounts cla ON cla.id=cli.credit_limit_account_id AND should_exclude_dpd_calculation
        INNER JOIN data-platform-indodana.vayu.indodana_athena_applications a ON a.orderId=cli.orderId
        INNER JOIN data-platform-indodana.vayu.indodana_credit_installment_requests ir on ir.credit_limit_account_id=cla.id
        INNER JOIN data-platform-indodana.vayu.indodana_credit_loan_contracts lc ON lc.installment_request_id=ir.id
    ) f ON f.id=lc.id
    , report_period rp
    WHERE lc.type IN ('PRODUCT_INSTALLMENT','PAYLATER')
    and lc.dataDate=rp.dm_period
    and ((date(lc.approved_date,"Asia/Jakarta")<date(lc.paid_off_date,"Asia/Jakarta") and date(lc.paid_off_date,"Asia/Jakarta")<snapshot_date) or lc.paid_off_date is null)
    and (
        (lc.status in ('ACTIVE') AND f.id IS NULL)
        OR
        (date(lc.paid_Off_Date,"Asia/Jakarta") >= rp.period and lc.status='FINISHED')
        OR
        (f.id IS NOT NULL AND date_add(date(lc.approved_Date,"Asia/Jakarta"),interval 1 DAY)<snapshot_date)
    )
)
, summary as (
    select 
        'ATH' as type,
        C.id as loan_contract_id,
        substr(replace(C.id,'-', ''),1,28) AS Contract_Code,
        '' AS Branch,
        CASE
           WHEN c.status = 'ACTIVE' THEN 1
           ELSE 2
        END AS Phase_Of_Contract,
        CASE
           WHEN c.status = 'WAITING_FOR_DISBURSEMENT' THEN 1
           WHEN c.status = 'ACTIVE'THEN 2
           WHEN c.status = 'FINISHED'
                AND c.maxDayPastDue = 0 THEN 4
           WHEN c.status = 'FINISHED'
                AND c.maxDayPastDue < 0 THEN 5
           WHEN c.status = 'FINISHED'
                AND c.maxDayPastDue >= 1 THEN 12
           ELSE 28
        END AS Contract_Status,
        1 AS Type_Of_Contract,
        11 AS Credit_Classification,
        CASE
           WHEN UPPER(a.loanPurpose) = 'RENOVASI RUMAH' THEN 113
           WHEN UPPER(a.loanPurpose) IN ('KONSUMTIF','KEBUTUHAN SEHARI-HARI','REKREASI ATAU JALAN-JALAN','PEMBELIAN ELEKTRONIK','CICILAN KENDARAAN') THEN 116
           WHEN UPPER(a.loanPurpose) = 'BIAYA PENDIDIKAN' THEN 14
           WHEN UPPER(a.loanPurpose) = 'PENGEMBANGAN USAHA' THEN 83
           WHEN UPPER(a.loanPurpose) = 'CICILAN RUMAH' THEN 109
           WHEN UPPER(a.loanPurpose) IN ('BIAYA PENGOBATAN ANAK','PENGGABUNGAN PINJAMAN') THEN 99
           ELSE 0
        END AS Purpose_Of_Financing,
        NULL AS Name_Of_Insured,
        CAST(CASE
           WHEN UPPER(a.currentCompanyIndustry) IN ('PEMERINTAHAN','PARIWISATA','KEUANGAN/BANK','HUKUM DAN PERPAJAKAN','ASURANSI/FOREX') THEN 1
           WHEN UPPER(a.currentCompanyIndustry) = 'PERKEBUNAN' THEN 22
           WHEN UPPER(a.currentCompanyIndustry) = 'PERTAMBANGAN' THEN 42
           WHEN UPPER(a.currentCompanyIndustry) = 'INDUSTRI/PABRIK' THEN 51
           WHEN UPPER(a.currentCompanyIndustry) = 'KONTRAKTOR/PROPERTI' THEN 109
           WHEN UPPER(a.currentCompanyIndustry) IN ('PERDAGANGAN','RETAIL') THEN 205
           WHEN UPPER(a.currentCompanyIndustry) = 'RESTORAN/BAR/CAFE' THEN 206
           WHEN UPPER(a.currentCompanyIndustry) = 'TRANSPORTASI' THEN 213
           WHEN UPPER(a.currentCompanyIndustry) = 'TELEKOMUNIKASI' THEN 215
           WHEN UPPER(a.currentCompanyIndustry) = 'KONSULTAN' THEN 216
           WHEN UPPER(a.currentCompanyIndustry) = 'JASA/PELAYANAN' THEN 224
           WHEN UPPER(a.currentCompanyIndustry) = 'SPESIALIS' THEN 226
           WHEN UPPER(a.currentCompanyIndustry) = 'KESEHATAN/KLINIK' THEN 227
           WHEN UPPER(a.currentCompanyIndustry) IS NULL THEN 0
        END AS STRING) AS Economic_Sector,
        66 AS Currency_Of_Contract,
        coalesce(c.interest_rate,0.96) AS Interest_Rate,
        1 AS Type_Of_Interest_Rate,
        c.principal AS Total_Facility_Amount,
        c.principal AS Total_Amount,
        c.principal AS Initial_Total_Amount,
        c.principal AS Total_Taken_Amount,
        CASE 
            WHEN c.status = 'FINISHED' THEN 0 
            WHEN c.dayPastDue<=0 THEN coalesce(c.balance,0)
            else coalesce(c.balanceWithInterest,0) + coalesce(c.lateFee,0) end AS Outstanding_Amount,
        CASE WHEN c.status = 'FINISHED' THEN 0 else coalesce(c.balance,0) end AS Principal_Balance,
        CASE 
            WHEN c.status='FINISHED' or c.dayPastDue<=0 then 0
            else c.balanceWithInterest + coalesce(c.lateFee,0)
        end as Past_Due_Amount,
        CASE
           WHEN c.status = 'FINISHED' or c.dayPastDue < 1 THEN 0
           ELSE c.dayPastDue
        END AS Past_Due_Days,
        CASE WHEN c.status='FINISHED' or c.dayPastDue<=0 then 0 else coalesce(c.balanceWithInterest,0)-coalesce(c.balance,0) end AS Past_Due_Interest,
        0 AS Interest_Arrears,
        '' AS Interest_Arrears_Frequency,
        CASE WHEN c.status = 'FINISHED' or c.dayPastDue<=0 THEN 0 else c.balance end AS Principal_Arrears,
        '' AS Principal_Arrears_Frequency,
        CASE WHEN c.status = 'FINISHED' or c.dayPastDue<=0 THEN 0 else coalesce(c.lateFee,0) end AS Penalty,
        '' AS Initial_Agreement_Number,
        
        case when dm_period < '2019-11-01' then format_timestamp("%Y-%m-%d",c.approvedDate) else format_timestamp("%Y-%m-%d",c.approvedDate,"Asia/Jakarta") end AS Initial_Agreement_Date,
        '' AS Last_Agreement_Number,
        '' AS Last_Agreement_Date,
        case when dm_period < '2019-11-01' then format_timestamp("%Y-%m-%d",c.approvedDate) else format_timestamp("%Y-%m-%d",c.approvedDate,"Asia/Jakarta") end AS Disbursement_Date,
        case when dm_period < '2019-11-01' then format_timestamp("%Y-%m-%d",c.approvedDate) else format_timestamp("%Y-%m-%d",c.approvedDate,"Asia/Jakarta") end AS Start_Date,
        '' AS Restructuring_Date,
        
        format_date("%Y-%m-%d",date_add(date(c.approvedDate,"Asia/Jakarta"),interval c.approvedLoanTenureInMonth month)) as Maturity_Date,
        case 
            when c.status='ACTIVE'then null
            when c.status='FINISHED' and c.paidOffDate is null then format_date("%Y-%m-%d",date_add(date(c.approvedDate),interval c.approvedLoanTenureInMonth month))
            when dm_period < '2019-11-01' then
                case 
                    when date(c.approvedDate) = date(c.paidOffDate) then format_date("%Y-%m-%d",date_add(date(c.paidOffDate),interval 1 day)) 
                    else format_date("%Y-%m-%d",date(c.paidOffDate)) 
                end
            else
                case 
                    when date(c.approvedDate,"Asia/Jakarta") = date(c.paidOffDate,"Asia/Jakarta") then format_date("%Y-%m-%d",date_add(date(c.paidOffDate,"Asia/Jakarta"),interval 1 day)) 
                    else format_date("%Y-%m-%d",date(c.paidOffDate,"Asia/Jakarta")) 
                end 
        end as Real_End_Date,
        '' AS Condition_Date,
        CASE
           WHEN c.dayPastDue < 1 THEN 1
           WHEN c.dayPastDue <= 90 THEN 21
           WHEN c.dayPastDue <= 120 THEN 22
           WHEN c.dayPastDue <= 180 THEN 23
           WHEN c.dayPastDue > 180 THEN 24
           ELSE 0
        END AS Negative_Status_Of_The_Contract,
        NULL AS Delinquency_Date,
        CASE
           WHEN c.dayPastDue > 180 THEN format_date("%Y-%m-%d",date_add(date(c.nextDueDate,"Asia/Jakarta"),interval 180 day))
           ELSE NULL
        END AS Default_Date,
        '' AS Default_Reason,
        '' AS Default_Reason_Description,
        2 AS Syndicated_Loan,
        format_date("%Y-%m-%d",snapshot_date) as snapshot_date
    from data-platform-indodana.vayu.indodana_athena_applications a 
    inner join contracts_ath c on a.orderId=c.applicationOrderId
    LEFT JOIN payment_current_period p on p.loan_contract_id=c.loan_contract_id
    , report_period rp
    where c.dataDate=rp.dm_period
    and a.applicantPersonalEmail not like '%indodana.com%'
    and a.applicantPersonalEmail not like '%cermati.com%'
    and a.orderId not like 'LOP-%'
    and ((date(c.approvedDate,"Asia/Jakarta")<date(c.paidOffDate,"Asia/Jakarta") and date(c.paidOffDate,"Asia/Jakarta")<snapshot_date) or c.paidOffDate is null)
    and (
        c.status in ('ACTIVE')
        OR
        (p.loan_contract_id is not null and c.status='FINISHED')
        OR 
        date(c.paidOffDate,"Asia/Jakarta") >= rp.period
    )
    AND date(c.approvedDate,"Asia/Jakarta")<snapshot_date
    AND a.userId is not null
    UNION ALL
    /*Data CLI*/
    select 
        'CLI' as type,
        C.loan_contract_id,
        substr(replace(C.loan_contract_id,'-', ''),1,28) AS Contract_Code,
        '' AS Branch,
        CASE
          WHEN c.status = 'ACTIVE' THEN 1
          ELSE 2
        END AS Phase_Of_Contract,
        CASE
            WHEN c.status IN ('CANCELLED','CHARGEBACK') THEN 8
          WHEN c.status = 'WAITING_FOR_DISBURSEMENT' THEN 1
          WHEN c.status = 'ACTIVE'THEN 2
          WHEN c.status = 'FINISHED'
                AND c.max_dpd = 0 THEN 4
          WHEN c.status = 'FINISHED'
                AND c.max_dpd < 0 THEN 5
          WHEN c.status = 'FINISHED'
                AND c.max_dpd >= 1 THEN 12
          ELSE 28
        END AS Contract_Status,
        1 AS Type_Of_Contract,
        11 AS Credit_Classification,
        CASE
          WHEN UPPER(a.loanPurpose) = 'RENOVASI RUMAH' THEN 113
          WHEN UPPER(a.loanPurpose) IN ('KONSUMTIF','KEBUTUHAN SEHARI-HARI','REKREASI ATAU JALAN-JALAN','PEMBELIAN ELEKTRONIK','CICILAN KENDARAAN') THEN 116
          WHEN UPPER(a.loanPurpose) = 'BIAYA PENDIDIKAN' THEN 14
          WHEN UPPER(a.loanPurpose) = 'PENGEMBANGAN USAHA' THEN 83
          WHEN UPPER(a.loanPurpose) = 'CICILAN RUMAH' THEN 109
          WHEN UPPER(a.loanPurpose) IN ('BIAYA PENGOBATAN ANAK','PENGGABUNGAN PINJAMAN') THEN 99
          ELSE 0
        END AS Purpose_Of_Financing,
        NULL AS Name_Of_Insured,
        CAST(CASE
          WHEN UPPER(a.currentCompanyIndustry) IN ('PEMERINTAHAN','PARIWISATA','KEUANGAN/BANK','HUKUM DAN PERPAJAKAN','ASURANSI/FOREX') THEN 1
          WHEN UPPER(a.currentCompanyIndustry) = 'PERKEBUNAN' THEN 22
          WHEN UPPER(a.currentCompanyIndustry) = 'PERTAMBANGAN' THEN 42
          WHEN UPPER(a.currentCompanyIndustry) = 'INDUSTRI/PABRIK' THEN 51
          WHEN UPPER(a.currentCompanyIndustry) = 'KONTRAKTOR/PROPERTI' THEN 109
          WHEN UPPER(a.currentCompanyIndustry) IN ('PERDAGANGAN','RETAIL') THEN 205
          WHEN UPPER(a.currentCompanyIndustry) = 'RESTORAN/BAR/CAFE' THEN 206
          WHEN UPPER(a.currentCompanyIndustry) = 'TRANSPORTASI' THEN 213
          WHEN UPPER(a.currentCompanyIndustry) = 'TELEKOMUNIKASI' THEN 215
          WHEN UPPER(a.currentCompanyIndustry) = 'KONSULTAN' THEN 216
          WHEN UPPER(a.currentCompanyIndustry) = 'JASA/PELAYANAN' THEN 224
          WHEN UPPER(a.currentCompanyIndustry) = 'SPESIALIS' THEN 226
          WHEN UPPER(a.currentCompanyIndustry) = 'KESEHATAN/KLINIK' THEN 227
          WHEN UPPER(a.currentCompanyIndustry) IS NULL THEN 0
        END AS STRING) AS Economic_Sector,
        66 AS Currency_Of_Contract,
        coalesce(c.interest_rate,0) AS Interest_Rate,
        1 AS Type_Of_Interest_Rate,
        principal AS Total_Facility_Amount,
        principal AS Total_Amount,
        principal AS Initial_Total_Amount,
        principal AS Total_Taken_Amount,
        CASE 
            WHEN c.status = 'FINISHED' THEN 0 
            WHEN c.dpd<=0 THEN coalesce(c.balance_principal,0)
            else coalesce(c.balance_principal,0) + coalesce(c.balance_interest,0) + coalesce(c.balance_latefee,0) end AS Outstanding_Amount,
        CASE WHEN c.status = 'FINISHED' THEN 0 else coalesce(c.balance_principal,0) end AS Principal_Balance,
        CASE 
            WHEN c.status='FINISHED' or c.dpd<=0 then 0
            else coalesce(c.balance_principal,0) + coalesce(c.balance_interest,0) + coalesce(c.balance_latefee,0)
        end as Past_Due_Amount,
        CASE
          WHEN c.status = 'FINISHED' or c.dpd < 1 THEN 0
          ELSE c.dpd
        END AS Past_Due_Days,
        CASE WHEN c.status='FINISHED' or c.dpd<=0 THEN 0 ELSE coalesce(c.balance_interest,0) END AS Past_Due_Interest,
        0 AS Interest_Arrears,
        '' AS Interest_Arrears_Frequency,
        CASE WHEN c.status='FINISHED' or c.dpd<=0 THEN 0 ELSE coalesce(c.balance_principal,0) END AS Principal_Arrears,
        '' AS Principal_Arrears_Frequency,
        CASE WHEN c.status = 'FINISHED' or c.dpd<=0 THEN 0 else coalesce(c.balance_latefee,0) end AS Penalty,
        '' AS Initial_Agreement_Number,
        
        format_timestamp("%Y-%m-%d",c.approved_Date,"Asia/Jakarta") AS Initial_Agreement_Date,
        '' AS Last_Agreement_Number,
        '' AS Last_Agreement_Date,
        format_timestamp("%Y-%m-%d",c.approved_Date,"Asia/Jakarta") AS Disbursement_Date,
        format_timestamp("%Y-%m-%d",c.approved_Date,"Asia/Jakarta") AS Start_Date,
        '' AS Restructuring_Date,
        
        format_date("%Y-%m-%d",date_add(date(c.approved_date,"Asia/Jakarta"),interval c.tenure month)) as Maturity_Date,
        case 
            when c.status IN ('CANCELLED','CHARGEBACK') then format_date("%Y-%m-%d",date_add(date(c.approved_Date,"Asia/Jakarta"),interval 1 DAY))
            when c.status='ACTIVE' then null
            when c.status='FINISHED' and c.paid_Off_Date is null then format_date("%Y-%m-%d",date_add(date(c.approved_Date),interval c.tenure month))
            when date(c.paid_Off_Date,"Asia/Jakarta")=date(c.approved_Date,"Asia/Jakarta") THEN format_date("%Y-%m-%d",date_add(date(c.approved_Date),interval 1 DAY))
            else format_date("%Y-%m-%d",date(c.paid_Off_Date,"Asia/Jakarta")) 
        end as Real_End_Date,
        '' AS Condition_Date,
        CASE
          WHEN c.dpd < 1 THEN 1
          WHEN c.dpd <= 90 THEN 21
          WHEN c.dpd <= 120 THEN 22
          WHEN c.dpd <= 180 THEN 23
          WHEN c.dpd > 180 THEN 24
          ELSE 0
        END AS Negative_Status_Of_The_Contract,
        NULL AS Delinquency_Date,
        CASE
          WHEN c.dpd > 180 THEN format_date("%Y-%m-%d",date_add(date(c.next_Due_Date,"Asia/Jakarta"),interval 180 day))
          ELSE NULL
        END AS Default_Date,
        '' AS Default_Reason,
        '' AS Default_Reason_Description,
        2 AS Syndicated_Loan,
        format_date("%Y-%m-%d",snapshot_date) as snapshot_date
    from data-platform-indodana.vayu.indodana_athena_applications a 
    inner join trx c on c.orderId=a.orderId
    where a.applicantPersonalEmail not like '%indodana.com%'
    and a.applicantPersonalEmail not like '%cermati.com%'
    AND a.userId is not null
)

select * 
    /* Code to know which error occurs*/
    , case 
        when Real_End_Date >= snapshot_date then 1 
        when Start_Date >= Real_End_Date then 2
        when Start_Date >= Maturity_Date then 3
        when Start_Date >= snapshot_date then 4
        when (Past_Due_Days>0 and Past_Due_Amount<=0) then 5
        when (Phase_Of_Contract=2 and Real_End_Date is null) then 6
        when (Contract_Status in (4,5,12) and (Penalty>0 or Outstanding_Amount>0 or Principal_Balance>0)) then 7
        when (Real_End_Date is not null and Phase_Of_Contract<>2) then 8
        when (Outstanding_Amount <> (Principal_Balance+Past_Due_Interest+Penalty)) then 9
        when (Past_Due_Amount <> (Principal_Arrears+Past_Due_Interest+Penalty)) then 10
        when (Past_Due_Days<=0 and Past_Due_Amount>0) then 11
    end as error_category
from summary
""".format(Period=param_date)
query_job = client.query(sql)
contractData = query_job.to_dataframe()

Get Data Individual to Dataframe Individual

In [5]:
client = bigquery.Client(project="athena-179008")
sql = """
    with report_period as (
    select 
        date_trunc(date('{Period}'),month) as period
        , case 
            when date('{Period}') < '2019-08-01' then date_add(date_trunc(date('{Period}'),month),interval 1 month)
            else date_add(date_add(date_trunc(date('{Period}'),month),interval 1 month),interval -1 day) 
        end as dm_period
        , date_add(date_trunc(date('{Period}'),month),interval 1 month) as snapshot_date
)
-- select * from report_period
/*CONTRACT ATH*/
, contracts_ath as (
    select 
        lc.dataDate
        , coalesce(c.applicationOrderId,a.orderId) as applicationOrderId
        , coalesce(c.id,lc.id) as id
        , lc.id as loan_contract_id
        , lc.status
        , lc.approved_date as approvedDate
        , lc.paid_off_date as paidOffDate
        , cast(lc.loan_amount as int64) as approvedLoanAmount
        , cast(lc.principal as int64) as principal
        , lc.tenure as approvedLoanTenureInMonth
        , coalesce(lcl.dpd,0) as dayPastDue
        , case when lcl.max_dpd_paid is null then lcl.dpd when lcl.dpd is null then lcl.max_dpd_paid else greatest(lcl.dpd,lcl.max_dpd_paid) end as maxDayPastDue
        , lcl.nextduedate
        , cast(lcl.balance as int64) as balance
        , cast(lcl.balanceWithInterest as int64) as balanceWithInterest
        , cast(lcl.latefee as int64) as latefee
        , lcl.last_due_date
        , lc.type
    from athena-179008.vayu_data_mart.indodana_credit_loan_contracts as lc
    inner join data-platform-indodana.vayu.indodana_credit_installment_requests as ir on lc.installment_request_id = ir.id
    inner join data-platform-indodana.vayu.indodana_athena_applications a on a.orderId=ir.entity_id
    LEFT join (
        select 
            dataDate
            , loan_contract_id
            , min(case when status in ('UNPAID','PARTIALLY_PAID') then due_date end) as nextduedate
            , date_diff(dataDate,date(min(case when status in ('UNPAID','PARTIALLY_PAID') then due_date end),"Asia/Jakarta"),day) as dpd
            , max(case when status = 'PAID' then date_diff(date(paid_off_date,"Asia/Jakarta"),date(due_date,"Asia/Jakarta"),day) end) max_dpd_paid
            , sum(case when type='PRINCIPAL' then balance else 0 end) as balance
            , sum(case when type IN ('INTEREST','PRINCIPAL') then balance else 0 end) as balanceWithInterest
            , sum(case when type='LATE_FEE' then balance else 0 end) as latefee
            , max(case when type='PRINCIPAL' then due_date end) as last_due_date
        from athena-179008.vayu_data_mart.indodana_credit_loan_contract_ledgers 
        where status not in ('CANCELLED')
        group by 1,2
    ) lcl on lcl.loan_contract_id=lc.id AND lc.dataDate=lcl.dataDate
    LEFT JOIN data-platform-indodana.vayu.indodana_credit_credit_rates r ON r.id=lc.credit_rate_id
    LEFT JOIN data-platform-indodana.vayu.indodana_athena_contracts c ON c.creditLoanContractId=lc.id
    WHERE ir.type IN ('CASH_LOAN')
)
/* Get All contractId that has payments during reporting period. */
, payment_current_period as (
    -- select p.contractId
    --     , max(paymentProviderReceivedDate) as max_payment_date
    -- from data-platform-indodana.vayu.indodana_athena_user_payments p
    -- where paymentProviderReceivedDate < (select max(approvedDate) from athena-179008.vayu_data_mart.indodana_athena_contracts where dataDate=(select dm_period from report_period))
    -- and date(paymentProviderReceivedDate,"Asia/Jakarta") >= (select period from report_period)
    -- and paymentProvider<>'WAIVER'
    -- group by 1
    select p.loan_contract_id
        , max(Provider_Received_Date) as max_payment_date
    from athena-179008.vayu_data_mart.view_ath_user_payments p
    where Provider_Received_Date < (select max(approvedDate) from contracts_ath where dataDate=(select dm_period from report_period))
    and date(Provider_Received_Date,"Asia/Jakarta") >= (select period from report_period)
    group by 1
)
/*Data CLI*/
, ledgers as (
    select 
        dataDate
        , loan_contract_id
        , min(case when status in ('UNPAID','PARTIALLY_PAID') then due_date end) as next_due_date
        , date_diff(dataDate,date(min(case when status in ('UNPAID','PARTIALLY_PAID') then due_date end),"Asia/Jakarta"),day) as dpd
        , max(case when status = 'PAID' then date_diff(date(paid_off_date,"Asia/Jakarta"),date(due_date,"Asia/Jakarta"),day) end) max_dpd_paid
        , sum(balance) as outstanding_balance
        , sum(case when type='PRINCIPAL' then balance else 0 end) as balance_principal
        , sum(case when type='INTEREST' then balance else 0 end) as balance_interest
        , sum(case when type='LATE_FEE' then balance else 0 end) as balance_latefee
    from athena-179008.vayu_data_mart.indodana_credit_loan_contract_ledgers 
    , report_period rp
    where status not in ('CANCELLED')
    and dataDate=rp.dm_period
    group by 1,2
)
, trx as (
    select 
        ir.borrower_id
        , a.OrderId
        , a.partner
        , ir.credit_limit_account_id
        , ir.entity_id
        , lc.id as loan_contract_id
        , CASE WHEN f.id IS NOT NULL then 'CANCELLED' else lc.status END as status
        , lc.tenure
        , lc.approved_date
        , lc.paid_off_date
        , cast(lc.loan_amount as int64) as loan_amount
        , cast(lc.principal as int64) as principal
        , cast(lc.interest as int64) as interest
        , cast((lc.principal+lc.interest)/lc.tenure as int64) as installment
        , lcl.next_due_date
        , coalesce(lcl.dpd,0) as dpd
        , case when lcl.max_dpd_paid is null then lcl.dpd when lcl.dpd is null then lcl.max_dpd_paid else greatest(lcl.dpd,lcl.max_dpd_paid) end as max_dpd
        , cast(lcl.outstanding_balance as int64) as outstanding_balance
        , cast(lcl.balance_principal as int64) as balance_principal
        , cast(lcl.balance_interest as int64) as balance_interest
        , cast(lcl.balance_latefee as int64) as balance_latefee
        , dm_period
        , snapshot_date
    from athena-179008.vayu_data_mart.indodana_credit_loan_contracts as lc
    inner join data-platform-indodana.vayu.indodana_credit_installment_requests as ir on lc.installment_request_id = ir.id
    inner join data-platform-indodana.vayu.indodana_credit_credit_limit_accounts cla on cla.id=ir.credit_limit_account_id
    LEFT join data-platform-indodana.vayu.indodana_athena_applications app
        ON coalesce(app.partner,'INDODANA')=cla.type
        AND app.userId=ir.borrower_id
        and substr(app.orderId,1,3)='CLI'
        and app.status='Approved'
    inner join data-platform-indodana.vayu.indodana_athena_applications a 
        on a.orderId=COALESCE(SPLIT(cla.idempotency_key,'_application.')[SAFE_OFFSET(1)],app.OrderId)
    inner join ledgers lcl on lcl.loan_contract_id=lc.id
    /* Check Fraud Victim Data */
    LEFT JOIN (
        SELECT
            lc.id, cla.updated_At
        FROM athena-179008.vayu_data_mart.indodanamarketing_cli_application cli
        INNER JOIN data-platform-indodana.vayu.indodana_credit_credit_limit_accounts cla ON cla.id=cli.credit_limit_account_id AND should_exclude_dpd_calculation
        INNER JOIN data-platform-indodana.vayu.indodana_athena_applications a ON a.orderId=cli.orderId
        INNER JOIN data-platform-indodana.vayu.indodana_credit_installment_requests ir on ir.credit_limit_account_id=cla.id
        INNER JOIN data-platform-indodana.vayu.indodana_credit_loan_contracts lc ON lc.installment_request_id=ir.id
    ) f ON f.id=lc.id
    , report_period rp
    WHERE lc.type IN ('PRODUCT_INSTALLMENT','PAYLATER')
    and lc.dataDate=rp.dm_period
    and ((date(lc.approved_date,"Asia/Jakarta")<date(lc.paid_off_date,"Asia/Jakarta") and date(lc.paid_off_date,"Asia/Jakarta")<snapshot_date) or lc.paid_off_date is null)
    and (
        (lc.status in ('ACTIVE') AND f.id IS NULL)
        OR
        (date(lc.paid_Off_Date,"Asia/Jakarta") >= rp.period and lc.status='FINISHED')
        OR
        (f.id IS NOT NULL AND date_add(date(lc.approved_Date,"Asia/Jakarta"),interval 1 DAY)<snapshot_date)
    )
)
, npwp_data AS (
    SELECT 
        DISTINCT N.userId, N.npwp
    FROM data-platform-indodana.vayu.indodana_athena_applications A
    INNER JOIN data-platform-indodana.vayu.indodana_athena_contracts C ON A.orderId = C.applicationOrderId
    INNER JOIN data-platform-indodana.vayu.indodana_athena_npwp_validations N ON A.userId = N.userId
    WHERE N.isValid = TRUE 
)
, summary as (
    select 
        substr(replace(a.userId,'-',''),1,28) AS Customer_Code,
        substr(replace(C.id,'-', ''),1,28) AS Contract_Code,
        1 AS Role_Of_Customer,
        regexp_replace(replace(replace(a.applicantName,',',' '),'|',' '),'\\t',' ') AS Full_Name,
        '' AS Alias,
        a.verifiedApplicantMothersMaidenName AS Mothers_Maiden_Name,
        1 AS Classification_Of_Individual,
        CASE
           WHEN a.applicantGender = 'MALE' THEN 1
           ELSE 2
        END AS Gender,
        date(a.applicantDateOfBirth) AS Date_Of_Birth,
        a.applicantPlaceOfBirthCity AS Place_Of_Birth,
        CASE
           WHEN a.typeOfJobs = 'Freelance' THEN 0
           WHEN a.typeOfJobs = 'Tidak Bekerja' THEN 3
           WHEN a.typeOfJobs ='Pelajar' THEN 1
           WHEN a.typeOfJobs = 'Ibu Rumah Tangga' THEN 5
           ELSE 2
        END AS Social_Status,
        97 AS Citizenship,
        '' AS Employment,
        replace(replace(regexp_replace(regexp_replace(regexp_replace((COALESCE(a.currentCompanyName, a.verifiedCurrentCompanyName)),'\\n',''),'\\r',''),'\\t',' '),',',' '),'|','') as Employer_Name,
        CAST(CASE
           WHEN a.currentCompanyIndustry IN ('Pemerintahan','Pariwisata','Keuangan/Bank','Hukum dan Perpajakan','Asuransi/Forex') THEN 1
           WHEN a.currentCompanyIndustry = 'Perkebunan' THEN 22
           WHEN a.currentCompanyIndustry = 'Pertambangan' THEN 42
           WHEN a.currentCompanyIndustry = 'Industri/Pabrik' THEN 51
           WHEN a.currentCompanyIndustry = 'Kontraktor/Properti' THEN 109
           WHEN a.currentCompanyIndustry IN ('Perdagangan','Retail') THEN 205
           WHEN a.currentCompanyIndustry = 'Restoran/Bar/Cafe' THEN 206
           WHEN a.currentCompanyIndustry = 'Transportasi' THEN 213
           WHEN a.currentCompanyIndustry = 'Telekomunikasi' THEN 215
           WHEN a.currentCompanyIndustry = 'Konsultan' THEN 216
           WHEN a.currentCompanyIndustry = 'Jasa/Pelayanan' THEN 224
           WHEN a.currentCompanyIndustry = 'Spesialis' THEN 226
           WHEN a.currentCompanyIndustry = 'Kesehatan/Klinik' THEN 227
           WHEN a.currentCompanyIndustry IS NULL THEN 0
        END AS STRING) AS Employer_Sector,
        CASE
           WHEN a.applicantLastEducationLevel = 'S1' THEN 9
           WHEN a.applicantLastEducationLevel = 'S2' THEN 10
           WHEN a.applicantLastEducationLevel = 'S3' THEN 11
           WHEN a.applicantLastEducationLevel = 'Akademi/D3' THEN 8
           WHEN a.applicantLastEducationLevel IS NULL THEN 0
           ELSE 99
        END AS Education,
        n.npwp AS NPWP,
        TRIM(replace(replace(a.applicantIdNumber,'O','0'),'o','0')) AS KTP,
        '' AS Passport_Number,
        97 AS Passport_Issuer_Country,
        substr(
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        replace(
                            replace(
                                replace(
                                    case 
                                        when coalesce(a.applicantCurrentAddress,'')<>'' then a.applicantCurrentAddress
                                        when coalesce(a.applicantAddress,'')<>'' then a.applicantAddress
                                        when coalesce(a.currentCompanyAddress,'')<>'' then a.currentCompanyAddress
                                        when coalesce(a.applicantCurrentResidenceVillage,'')<>'' then a.applicantCurrentResidenceVillage
                                        when coalesce(a.applicantCurrentResidenceDistrict,'')<>'' then a.applicantCurrentResidenceDistrict
                                        when coalesce(a.applicantCurrentResidenceCity,'')<>'' then a.applicantCurrentResidenceCity
                                        when coalesce(a.applicantResidenceVillage,'')<>'' then a.applicantResidenceVillage
                                        when coalesce(a.applicantResidenceDistrict,'')<>'' then a.applicantResidenceDistrict
                                        when coalesce(a.applicantResidenceCity,'')<>'' then a.applicantResidenceCity
                                        when coalesce(a.applicantCurrentResidenceProvince,'')<>'' then a.applicantCurrentResidenceProvince
                                        when coalesce(a.applicantResidenceProvince,'')<>'' then a.applicantResidenceProvince
                                    end
    		                    ,',',' ')
    		                ,';',' ')
    		            ,'|',' ')
    		        ,'\\n',' ')
    		    ,'\\r',' ')
    		,'\\t',' ')
    	, 1, 120) as Street,
        SUBSTR(CASE
           WHEN a.addressIsCurrentAddress = 'Ya' THEN a.applicantResidenceCity
           ELSE a.applicantCurrentResidenceCity
        END,1,60) AS City,
        SUBSTR(CASE
           WHEN a.addressIsCurrentAddress = 'Ya' THEN a.applicantResidencePostalCode
           ELSE a.applicantCurrentResidencePostalCode
        END,1,8) AS Postal_Code,
        SUBSTR(CASE
           WHEN a.addressIsCurrentAddress = 'Ya' THEN a.applicantResidenceDistrict
           ELSE a.applicantCurrentResidenceDistrict
        END,1,60) AS District,
        SUBSTR(CASE
           WHEN a.addressIsCurrentAddress = 'Ya' THEN a.applicantResidenceVillage
           ELSE a.applicantCurrentResidenceVillage
        END,1,60) AS Parish,
        97 AS Country,
        '' AS Address_Line,
        replace(a.applicantMobilePhoneNumber1,'+62','0') AS Mobile_Phone,
        '' AS Fixed_Line
    from data-platform-indodana.vayu.indodana_athena_applications a 
    inner join contracts_ath c on a.orderId=c.applicationOrderId
    -- inner join data-platform-indodana.vayu.indodana_athena_contracts cn on cn.id=c.id
    -- inner join data-platform-indodana.vayu.indodana_athena_rates r on r.id=cn.rateId
    -- inner join data-platform-indodana.vayu.indodana_athena_installment_commitments ic on ic.contractId=c.id
    LEFT JOIN payment_current_period p on p.loan_contract_id=c.loan_contract_id
    LEFT JOIN npwp_data n on n.userId=a.UserId
    , report_period rp
    where c.dataDate=rp.dm_period
    and a.applicantPersonalEmail not like '%indodana.com%'
    and a.applicantPersonalEmail not like '%cermati.com%'
    and a.orderId not like 'LOP-%'
    and ((date(c.approvedDate,"Asia/Jakarta")<date(c.paidOffDate,"Asia/Jakarta") and date(c.paidOffDate,"Asia/Jakarta")<snapshot_date) or c.paidOffDate is null)
    and (
        c.status in ('ACTIVE')
        OR
        (p.loan_contract_id is not null and c.status='FINISHED')
        OR 
        date(c.paidOffDate,"Asia/Jakarta") >= rp.period
    )
    AND date(c.approvedDate,"Asia/Jakarta")<snapshot_date
    AND a.userId is not null
    
    
    -- and (c.status in ('ACTIVE') or (c.status in ('FINISHED') and date(c.paidOffDate) < snapshot_date) or (c.status in ('FINISHED') and c.paidOffDate is null))
    
    UNION ALL
    
    /*DATA CLI*/
    select 
        substr(replace(a.userId,'-',''),1,28) AS Customer_Code,
        substr(replace(C.loan_contract_id,'-', ''),1,28) AS Contract_Code,
        1 AS Role_Of_Customer,
        regexp_replace(replace(replace(a.applicantName,',',' '),'|',' '),'\\t',' ') AS Full_Name,
        '' AS Alias,
        a.verifiedApplicantMothersMaidenName AS Mothers_Maiden_Name,
        1 AS Classification_Of_Individual,
        CASE
           WHEN a.applicantGender = 'MALE' THEN 1
           ELSE 2
        END AS Gender,
        case when a.orderId='CLI-P1ZXJFGY' then date('1988-10-07') else date(coalesce(a.applicantDateOfBirth,d.dob)) end AS Date_Of_Birth,
        a.applicantPlaceOfBirthCity AS Place_Of_Birth,
        CASE
           WHEN a.typeOfJobs = 'Freelance' THEN 0
           WHEN a.typeOfJobs = 'Tidak Bekerja' THEN 3
           WHEN a.typeOfJobs ='Pelajar' THEN 1
           WHEN a.typeOfJobs = 'Ibu Rumah Tangga' THEN 5
           ELSE 2
        END AS Social_Status,
        97 AS Citizenship,
        '' AS Employment,
        replace(replace(regexp_replace(regexp_replace(regexp_replace((COALESCE(a.currentCompanyName, a.verifiedCurrentCompanyName)),'\\n',''),'\\r',''),'\\t',' '),',',' '),'|','') as Employer_Name,
        CAST(CASE
           WHEN a.currentCompanyIndustry IN ('Pemerintahan','Pariwisata','Keuangan/Bank','Hukum dan Perpajakan','Asuransi/Forex') THEN 1
           WHEN a.currentCompanyIndustry = 'Perkebunan' THEN 22
           WHEN a.currentCompanyIndustry = 'Pertambangan' THEN 42
           WHEN a.currentCompanyIndustry = 'Industri/Pabrik' THEN 51
           WHEN a.currentCompanyIndustry = 'Kontraktor/Properti' THEN 109
           WHEN a.currentCompanyIndustry IN ('Perdagangan','Retail') THEN 205
           WHEN a.currentCompanyIndustry = 'Restoran/Bar/Cafe' THEN 206
           WHEN a.currentCompanyIndustry = 'Transportasi' THEN 213
           WHEN a.currentCompanyIndustry = 'Telekomunikasi' THEN 215
           WHEN a.currentCompanyIndustry = 'Konsultan' THEN 216
           WHEN a.currentCompanyIndustry = 'Jasa/Pelayanan' THEN 224
           WHEN a.currentCompanyIndustry = 'Spesialis' THEN 226
           WHEN a.currentCompanyIndustry = 'Kesehatan/Klinik' THEN 227
           WHEN a.currentCompanyIndustry IS NULL THEN 0
        END AS STRING) AS Employer_Sector,
        CASE
           WHEN a.applicantLastEducationLevel = 'S1' THEN 9
           WHEN a.applicantLastEducationLevel = 'S2' THEN 10
           WHEN a.applicantLastEducationLevel = 'S3' THEN 11
           WHEN a.applicantLastEducationLevel = 'Akademi/D3' THEN 8
           WHEN a.applicantLastEducationLevel IS NULL THEN 0
           ELSE 99
        END AS Education,
        n.npwp AS NPWP,
        TRIM(replace(replace(a.applicantIdNumber,'O','0'),'o','0')) AS KTP,
        '' AS Passport_Number,
        97 AS Passport_Issuer_Country,
        substr(
            regexp_replace(
                regexp_replace(
                    regexp_replace(
                        replace(
                            replace(
                                replace(
                                    case 
                                        when coalesce(a.applicantCurrentAddress,'')<>'' then a.applicantCurrentAddress
                                        when coalesce(a.applicantAddress,'')<>'' then a.applicantAddress
                                        when coalesce(a.currentCompanyAddress,'')<>'' then a.currentCompanyAddress
                                        when coalesce(a.applicantCurrentResidenceVillage,'')<>'' then a.applicantCurrentResidenceVillage
                                        when coalesce(a.applicantCurrentResidenceDistrict,'')<>'' then a.applicantCurrentResidenceDistrict
                                        when coalesce(a.applicantCurrentResidenceCity,'')<>'' then a.applicantCurrentResidenceCity
                                        when coalesce(a.applicantResidenceVillage,'')<>'' then a.applicantResidenceVillage
                                        when coalesce(a.applicantResidenceDistrict,'')<>'' then a.applicantResidenceDistrict
                                        when coalesce(a.applicantResidenceCity,'')<>'' then a.applicantResidenceCity
                                        when coalesce(a.applicantCurrentResidenceProvince,'')<>'' then a.applicantCurrentResidenceProvince
                                        when coalesce(a.applicantResidenceProvince,'')<>'' then a.applicantResidenceProvince
                                    end
    		                    ,',',' ')
    		                ,';',' ')
    		            ,'|',' ')
    		        ,'\\n',' ')
    		    ,'\\r',' ')
    		,'\\t',' ')
    	, 1, 120) as Street,
        SUBSTR(CASE
           WHEN a.addressIsCurrentAddress = 'Ya' THEN a.applicantResidenceCity
           ELSE a.applicantCurrentResidenceCity
        END,1,60) AS City,
        SUBSTR(CASE
           WHEN a.addressIsCurrentAddress = 'Ya' THEN a.applicantResidencePostalCode
           ELSE a.applicantCurrentResidencePostalCode
        END,1,8) AS Postal_Code,
        SUBSTR(CASE
           WHEN a.addressIsCurrentAddress = 'Ya' THEN a.applicantResidenceDistrict
           ELSE a.applicantCurrentResidenceDistrict
        END,1,60) AS District,
        SUBSTR(CASE
           WHEN a.addressIsCurrentAddress = 'Ya' THEN a.applicantResidenceVillage
           ELSE a.applicantCurrentResidenceVillage
        END,1,60) AS Parish,
        97 AS Country,
        '' AS Address_Line,
        replace(a.applicantMobilePhoneNumber1,'+62','0') AS Mobile_Phone,
        '' AS Fixed_Line
    from data-platform-indodana.vayu.indodana_athena_applications a 
    INNER JOIN TRX c on c.orderId=a.orderId
    LEFT JOIN npwp_data n on n.userId=a.UserId
    /* To fill in orderId's without DOB information */
    LEFT JOIN (
        SELECT 
            userId
            , MAX(applicantDateOfBirth) as dob
        FROM data-platform-indodana.vayu.indodana_athena_applications
        WHERE REGEXP_CONTAINS(status,'(?i)(Approve|Reject|Not Qualified)')
        GROUP BY 1
    ) d ON d.userId=a.userId
    where a.applicantPersonalEmail not like '%indodana.com%'
    and a.applicantPersonalEmail not like '%cermati.com%'
    AND a.userId is not null
)

select *
    /* Code to know which error occurs*/
    , case 
        when length(street)>128 then 1 
        when (coalesce(Street,'')='' and coalesce(Address_Line,'')='') then 2
        when length(Customer_Code)>28 then 3
        when length(KTP)<>16 then 4
        when length(Postal_Code)>8 then 5
        when (safe_cast(KTP as int64) is null) then 6
        when length(District)>64 or length(Parish)>64 or length(City)>64 then 7
        when Date_Of_Birth is null then 8
    end as error_category
from summary
""".format(Period=param_date)
query_job = client.query(sql)
individual = query_job.to_dataframe()

In [6]:
individual.dtypes

Customer_Code                   object
Contract_Code                   object
Role_Of_Customer                 int64
Full_Name                       object
Alias                           object
Mothers_Maiden_Name             object
Classification_Of_Individual     int64
Gender                           int64
Date_Of_Birth                   object
Place_Of_Birth                  object
Social_Status                    int64
Citizenship                      int64
Employment                      object
Employer_Name                   object
Employer_Sector                 object
Education                        int64
NPWP                            object
KTP                             object
Passport_Number                 object
Passport_Issuer_Country          int64
Street                          object
City                            object
Postal_Code                     object
District                        object
Parish                          object
Country                  

In [7]:
contractData.dtypes

type                               object
loan_contract_id                   object
Contract_Code                      object
Branch                             object
Phase_Of_Contract                   int64
Contract_Status                     int64
Type_Of_Contract                    int64
Credit_Classification               int64
Purpose_Of_Financing                int64
Name_Of_Insured                    object
Economic_Sector                    object
Currency_Of_Contract                int64
Interest_Rate                      object
Type_Of_Interest_Rate               int64
Total_Facility_Amount               int64
Total_Amount                        int64
Initial_Total_Amount                int64
Total_Taken_Amount                  int64
Outstanding_Amount                  int64
Principal_Balance                   int64
Past_Due_Amount                     int64
Past_Due_Days                       int64
Past_Due_Interest                   int64
Interest_Arrears                  

Check Invalid Data

In [8]:
# import pandas as pd
# pd.set_option('display.max_colwidth', None)
# print(individual['Street'][individual['Contract_Code']=='23b6134e8b864c41b441fda77384'].str.len())
# individual['Street'][individual['Contract_Code']=='23b6134e8b864c41b441fda77384']
# 23b6134e8b864c41b441fda77384

In [9]:
print(
    'Total Rows not valid : \n' +
    str(individual['error_category'].notnull().values.sum()) + ' rows from Individual data\n' + 
    str(contractData['error_category'].notnull().values.sum()) + ' rows from Contract Data'
)

Total Rows not valid : 
0 rows from Individual data
0 rows from Contract Data


See Invalid data detail

In [10]:
contractData[contractData['error_category'].notnull().values]

,type,loan_contract_id,Contract_Code,Branch,Phase_Of_Contract,Contract_Status,Type_Of_Contract,Credit_Classification,Purpose_Of_Financing,Name_Of_Insured,...,Real_End_Date,Condition_Date,Negative_Status_Of_The_Contract,Delinquency_Date,Default_Date,Default_Reason,Default_Reason_Description,Syndicated_Loan,snapshot_date,error_category


In [11]:
# Change value at specific cells -> .at[Index,'ColumnName']=newvalue
# This code below is fo 2021-08 data, where balance=0, but status still unpaid, so we change the dpd value manually
# contractData.at[534112, 'Past_Due_Days'] = 0
# contractData[contractData['loan_contract_id']=='42a44ab1-f0fc-11eb-a1a7-b9a38990bd31']['Past_Due_Days']


In [12]:
individual[individual['error_category'].notnull().values]

,Customer_Code,Contract_Code,Role_Of_Customer,Full_Name,Alias,Mothers_Maiden_Name,Classification_Of_Individual,Gender,Date_Of_Birth,Place_Of_Birth,...,Street,City,Postal_Code,District,Parish,Country,Address_Line,Mobile_Phone,Fixed_Line,error_category


Exclude column error_category

In [13]:
del individual['error_category']
del contractData['error_category']
del contractData['type']
del contractData['loan_contract_id']
del contractData['snapshot_date']

In [14]:
individual.head()

,Customer_Code,Contract_Code,Role_Of_Customer,Full_Name,Alias,Mothers_Maiden_Name,Classification_Of_Individual,Gender,Date_Of_Birth,Place_Of_Birth,...,Passport_Issuer_Country,Street,City,Postal_Code,District,Parish,Country,Address_Line,Mobile_Phone,Fixed_Line
0,122dfc4608fc449282425ed86e49,e6b1c47b360c11ec928d9da41fc4,1,stanley hendrik,,None,1,1,1987-04-18,None,...,97,JL.17 AGUSTUS II NO.22 A KELURAHAN CIBANGKONG...,KOTA BANDUNG,40273,KECAMATAN BATUNUNGGAL,KELURAHAN CIBANGKONG,97,,087720000922,
1,122dfc4608fc449282425ed86e49,d3e41ce1362311ecae49c7a58379,1,stanley hendrik,,None,1,1,1987-04-18,None,...,97,JL.17 AGUSTUS II NO.22 A KELURAHAN CIBANGKONG...,KOTA BANDUNG,40273,KECAMATAN BATUNUNGGAL,KELURAHAN CIBANGKONG,97,,087720000922,
2,23b05d29d1ec482fb07aca82f072,d9a0adfc388311ecab8fd9d1bc97,1,Michael angelo yusuf,,None,1,1,1993-04-07,None,...,97,JL. TANAH TINGGI GG III NO0. 28 E KELURAHAN T...,KOTA JAKARTA PUSAT,10540,KECAMATAN JOHAR BARU,KELURAHAN TANAH TINGGI,97,,081806250810,
3,23b05d29d1ec482fb07aca82f072,d0f555e62d7711ec999353cabf8a,1,Michael angelo yusuf,,None,1,1,1993-04-07,None,...,97,JL. TANAH TINGGI GG III NO0. 28 E KELURAHAN T...,KOTA JAKARTA PUSAT,10540,KECAMATAN JOHAR BARU,KELURAHAN TANAH TINGGI,97,,081806250810,
4,1067d9071d3c4fdc91b627432514,54c5ef38da3211eb99350faf70a6,1,Prima Sulistiya,,None,1,1,1978-05-29,None,...,97,Jln.Prof Dr. Hamka Gg. Nilam 3 No 21 RT 02 RW ...,KOTA PONTIANAK,78113,KECAMATAN PONTIANAK KOTA,KELURAHAN SUNGAIJAWI,97,,082253097883,


In [15]:
contractData.head()

,Contract_Code,Branch,Phase_Of_Contract,Contract_Status,Type_Of_Contract,Credit_Classification,Purpose_Of_Financing,Name_Of_Insured,Economic_Sector,Currency_Of_Contract,...,Restructuring_Date,Maturity_Date,Real_End_Date,Condition_Date,Negative_Status_Of_The_Contract,Delinquency_Date,Default_Date,Default_Reason,Default_Reason_Description,Syndicated_Loan
0,ec82cb5c18c8488c8db2ccf59f3e,,1,2,1,11,113,None,51,66,...,,2021-02-05,None,,24,None,2021-07-04,,,2
1,59162f46195f4a3a9e7daca54e30,,1,2,1,11,14,None,224,66,...,,2019-10-07,None,,24,None,2020-04-04,,,2
2,4f529e683c7e4cb19f0678924b26,,1,2,1,11,116,None,213,66,...,,2019-10-09,None,,24,None,2020-04-06,,,2
3,ed352d4b2ce24c0491387b1389a7,,1,2,1,11,116,None,224,66,...,,2021-11-30,None,,1,None,None,,,2
4,6bf83877fa8a4687a19938f4c063,,1,2,1,11,116,None,227,66,...,,2022-03-03,None,,1,None,None,,,2


Check total Rows

In [16]:
if len(individual.index)==len(contractData.index) : print(
    'Individual - Contract Data : MATCH\n' + 
    str(len(individual.index)) + ' - ' + str(len(contractData.index))
)
else : print(
    'Individual - Contract Data : NOT MATCH\n' + 
    str(len(individual.index)) + ' - ' + str(len(contractData.index))
)

Individual - Contract Data : MATCH
956191 - 956191


Replace char ('|',',') with space 

In [17]:
individual = individual.replace('\|','', regex=True)
individual = individual.replace('\,','', regex=True)
# This code below works for 1 column
# individual['District']=individual['District'].str.replace('|','', regex=False)

In [18]:
# CHECK CERTAIN ROW
# Customer_Code
# 98c0b7e720664e7c97da6d932edc
# Contract_Code
# e739dab081a411eba84e91dfaa01
# ['District']
# individual['District'][(individual['Customer_Code']=='98c0b7e720664e7c97da6d932edc') & (individual['Contract_Code']=='e739dab081a411eba84e91dfaa01')]

In [19]:
individual[individual['Street'].str.len()>128]

,Customer_Code,Contract_Code,Role_Of_Customer,Full_Name,Alias,Mothers_Maiden_Name,Classification_Of_Individual,Gender,Date_Of_Birth,Place_Of_Birth,...,Passport_Issuer_Country,Street,City,Postal_Code,District,Parish,Country,Address_Line,Mobile_Phone,Fixed_Line


In [20]:
individual[individual['District'].str.contains('|',regex=False)==True]

,Customer_Code,Contract_Code,Role_Of_Customer,Full_Name,Alias,Mothers_Maiden_Name,Classification_Of_Individual,Gender,Date_Of_Birth,Place_Of_Birth,...,Passport_Issuer_Country,Street,City,Postal_Code,District,Parish,Country,Address_Line,Mobile_Phone,Fixed_Line


In [21]:
individual[individual['City'].str.contains(',',regex=False)==True]

,Customer_Code,Contract_Code,Role_Of_Customer,Full_Name,Alias,Mothers_Maiden_Name,Classification_Of_Individual,Gender,Date_Of_Birth,Place_Of_Birth,...,Passport_Issuer_Country,Street,City,Postal_Code,District,Parish,Country,Address_Line,Mobile_Phone,Fixed_Line


Check total Rows

In [22]:
if len(individual.index)==len(contractData.index) : print(
    'Individual - Contract Data : MATCH\n' + 
    str(len(individual.index)) + ' - ' + str(len(contractData.index))
)
else : print(
    'Individual - Contract Data : NOT MATCH\n' + 
    str(len(individual.index)) + ' - ' + str(len(contractData.index))
)

Individual - Contract Data : MATCH
956191 - 956191


Create New Directory if not exists

In [23]:
from pathlib import Path
new_directory = '01050100010_' + param_date[0:7].replace('-', '') + '_M_' + version + '_CSV'
Path(new_directory).mkdir(parents=True, exist_ok=True)
print(new_directory)

01050100010_202110_M_1_CSV


Export to csv UTF-8

In [24]:
contractData.to_csv(new_directory + '/ContractData.csv', sep = '|',index=False, encoding='utf-8-sig')
individual.to_csv(new_directory + '/Individual.csv', sep = '|',index=False, encoding='utf-8-sig')

# individual.to_csv(new_directory + '/Individual00.csv',index=False)

<b>Step to creating ContractData & Individual File is DONE.</b>

<h3>Now Creating Header File</h3>

In [25]:
import csv
with open(new_directory + '\Header.csv', 'w', newline='', encoding='utf-8-sig') as file:
    writer = csv.writer(file)
    writer.writerow(["BatchIdentifier"])
    writer.writerow([new_directory])

<h2>Copy Additional File to Folder</h2>

<h4>Copy one by one</h4>

In [26]:
# import shutil

# original = 'File/Collateral.csv'
# target = new_directory + '/Collateral.csv'
# shutil.copyfile(original, target)


# original = 'File/Company.csv'
# target = new_directory + '/Company.csv'
# shutil.copyfile(original, target)


# original = 'File/SubjectRelation.csv'
# target = new_directory + '/SubjectRelation.csv'
# shutil.copyfile(original, target)

<h4>copy all files at once</h4>

In [27]:
import os
import shutil
src = 'File'
dest = new_directory
src_files = os.listdir(src)
for file_name in src_files:
    full_file_name = os.path.join(src, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, dest)

<h3>Zip the Folder</h3>

In [33]:
from zipfile import ZipFile
import os
from os.path import basename

zipfile_name = new_directory + '.zip'
dirName = new_directory

# create a ZipFile object
with ZipFile(zipfile_name, 'w') as zipObj:
   # Iterate over all the files in directory
   for folderName, subfolders, filenames in os.walk(dirName):
       for filename in filenames:
           #create complete filepath of file in directory
           filePath = os.path.join(folderName, filename)
           # Add file to zip
           zipObj.write(filePath, basename(filePath))

In [34]:
# ZIP File one by  one

# Method 1 
# # create a ZipFile object
# zipObj = ZipFile('sample.zip', 'w')
# # Add multiple files to the zip
# zipObj.write('sample_file.csv')
# zipObj.write('test_1.log')
# zipObj.write('test_2.log')
# # close the Zip File
# zipObj.close()

# Method 2
# # Create a ZipFile Object
# with ZipFile('sample2.zip', 'w') as zipObj2:
#    # Add multiple files to the zip
#    zipObj2.write('sample_file.csv')
#    zipObj2.write('test_1.log')
#    zipObj2.write('test_2.log')